<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-up-NB" data-toc-modified-id="Set-up-NB-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set up NB</a></span></li><li><span><a href="#Initialize" data-toc-modified-id="Initialize-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Initialize</a></span><ul class="toc-item"><li><span><a href="#global-settings" data-toc-modified-id="global-settings-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>global settings</a></span></li><li><span><a href="#create-instruments" data-toc-modified-id="create-instruments-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>create instruments</a></span></li><li><span><a href="#Default-(instrument)-settings" data-toc-modified-id="Default-(instrument)-settings-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Default (instrument) settings</a></span></li></ul></li><li><span><a href="#Resonator-Spectroscopy" data-toc-modified-id="Resonator-Spectroscopy-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Resonator Spectroscopy</a></span></li></ul></div>

# Set up NB

In [1]:
%gui qt
%matplotlib qt

import sys
import os
import time
import logging
from importlib import reload

import numpy as np
from matplotlib import pyplot as plt

In [2]:
import qcodes as qc
import broadbean as bb

In [3]:
from pytopo.mplplots import init_nb_plotting; reload(init_nb_plotting)
from pytopo.mplplots.init_nb_plotting import *
from pytopo.mplplots import plots as mplplots

In [4]:
from pytopo.qctools import instruments as instools; reload(instools)
from pytopo.qctools.instruments import create_inst, add2station
from pytopo.qctools.dataset2 import select_experiment
from pytopo.qctools.measurement import MeasurementExt

from pytopo.rf import alazar_detector; reload(alazar_detector)
from pytopo.rf.alazar_detector import AlazarDetector

from plottr import qcodes_dataset; reload(qcodes_dataset)
from plottr.qcodes_dataset import QcodesDatasetSubscriber

from cqed import broadbean; reload(broadbean)
from cqed.broadbean import BroadBeanSequence, BluePrints

from cqed import bb_sequences; reload(bb_sequences)
from cqed.bb_sequences import TriggeredReadoutSequence

# Initialize

## global settings

In [5]:
SAMPLE = 'Cq334'

qc.config['core']['db_location'] = r"d:/OneDrive/BF1/Data/experiments.db"

BroadBeanSequence.chan_map = {
    1 : ['awg_voltage', 'ro_trigger', None],
}
BroadBeanSequence.chan_settings[1]['Vpp'] = 2.0
BroadBeanSequence.chan_settings[2]['Vpp'] = 2.0

BroadBeanSequence.sweep_wait = None

TriggeredReadoutSequence.chan_map = {
    1 : ['FILL.1', 'ro_trigger', None],
}

qc.dataset.database.initialise_database()

## create instruments

In [6]:
inst_list = []

# Alazar
# from qcodes.instrument_drivers.AlazarTech import utils; reload(utils)
# from qcodes.instrument_drivers.AlazarTech import ATS9360; reload(ATS9360)
from qcodes.instrument_drivers.AlazarTech.ATS9360 import AlazarTech_ATS9360
alazar = instools.create_inst(AlazarTech_ATS9360, 'alazar', force_new_instance=True)
inst_list.append(alazar)

# Alazar aquisition controllers
from pytopo.rf import alazar_acquisition; reload(alazar_acquisition)
from pytopo.rf.alazar_acquisition import RawAcqCtl, AvgDemodCtl, AvgIQCtl

raw_acq = instools.create_inst(RawAcqCtl, 'raw_acq', 'alazar', force_new_instance=True)
inst_list.append(raw_acq)

avgdemod_acq = instools.create_inst(AvgDemodCtl, 'avgdemod_acq', 'alazar', force_new_instance=True)
inst_list.append(avgdemod_acq)

avgiq_acq = instools.create_inst(AvgIQCtl, 'avgiq_acq', 'alazar', force_new_instance=True)
inst_list.append(avgiq_acq)

# RF sources
from qcodes.instrument_drivers.rohde_schwarz.SGS100A import RohdeSchwarz_SGS100A
LO = instools.create_inst(RohdeSchwarz_SGS100A, 'LO', address="TCPIP0::169.254.234.107")
RF = instools.create_inst(RohdeSchwarz_SGS100A, 'RF', address="TCPIP0::169.254.2.20")
inst_list.append(LO)
inst_list.append(RF)

from pytopo.rf.sources import HeterodyneSource
hetsrc = instools.create_inst(HeterodyneSource, 'hetsrc', RF=RF, LO=LO, force_new_instance=True)
inst_list.append(hetsrc)

# AWG and sequences
from qcodes.instrument_drivers.tektronix.AWG5014 import Tektronix_AWG5014
awg = instools.create_inst(Tektronix_AWG5014, 'awg', address='TCPIP0::169.254.183.196::inst0::INSTR')
inst_list.append(awg)

from qcodes.instrument_drivers.Minicircuits.RUDAT_13G_90 import RUDAT_13G_90_USB
ro_attenuator = instools.create_inst(RUDAT_13G_90_USB, 'ro_attenuator')
inst_list.append(ro_attenuator)






station = qc.Station(*inst_list)

Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110524, firmware:3.1.19.15-3.50.124.67) in 0.08s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110525, firmware:3.1.19.15-3.50.124.67) in 0.04s
Connected to: TEKTRONIX AWG5014C (serial:B051388, firmware:SCPI:99.0 FW:4.6.0.7) in 0.07s
Connected to: Mini-Circuits MN=RUDAT-13G-90 (serial:SN=11704210002, firmware:C9-2) in 0.61s


## Default (instrument) settings

In [7]:
with alazar.syncing():
    alazar.clock_source('INTERNAL_CLOCK')
    alazar.sample_rate(int(0.2e9))
    alazar.clock_edge('CLOCK_EDGE_RISING')
    alazar.external_sample_rate(int(1e9))
    alazar.decimation(1)
    alazar.coupling1('DC')
    alazar.coupling2('DC')
    alazar.channel_range1(0.4)
    alazar.channel_range2(0.4)
    alazar.impedance1(50)
    alazar.impedance2(50)
    alazar.trigger_source1('EXTERNAL')
    alazar.trigger_level1(128 + 5)
    alazar.external_trigger_coupling('DC')
    alazar.external_trigger_range('ETR_5V')
    alazar.trigger_delay(0)
    alazar.timeout_ticks(int(1e7))
    
    
hetsrc.frequency(500e6)
hetsrc.IF(8e6)
hetsrc.RF.on()
hetsrc.LO.on()
hetsrc.RF.power(0)
hetsrc.LO.power(15)
RF.pulsemod_state('off')

ro_attenuator.attenuation(30)

d:\onedrive\bf1\code\qcodes\qcodes\instrument_drivers\AlazarTech\ATS.py:564: UserWarning: Using Internal clock but parameter external_sample_rate is set.This will have no effect and is ignored
  warnings.warn("Using Internal clock but parameter external_sample_rate is set."


# Resonator Spectroscopy

In [8]:
### User settings

rf_frqs = np.linspace(200e6, 600e6, 401)[:] # spec frequencies
ro_atts = np.arange(26, 42, 2)
setup_awg = True
int_time = 10e-3
buffer_len = 100e-6

### End User settings



### Setting up instruments

# set IQ acquisition controller
acq = avgiq_acq
acq.trigger_func = None

# create new instance of sequence and detector
sequence = create_inst(TriggeredReadoutSequence, 'sequence', awg=awg, force_new_instance=True)
station = add2station(station, sequence)
detector = create_inst(AlazarDetector, name='detector', acqctl=acq, force_new_instance=True)
station = add2station(station, detector)

# configure the sequence parameters
sequence.pre_trigger_delay(1e-6) # some delay at the beginning
sequence.trigger_len(100e-9) # short trigger for the alazar
sequence.seq_len(10e-6) # quick rep-rate is OK here
if setup_awg:
    sequence.setup(program_awg=True, start_awg=True) # program and start only if something else is running before
else:
    sequence.setup(program_awg=False, start_awg=False)
    
# configure the Alazar detector
detector.set_sweeper(sequence) # set the sequence (it's just one dummy sweep-point here)
detector.demod_frq(hetsrc.IF()) # demod frequency from the sources
detector.acq_time(buffer_len) # not too long integration time ber buffer
detector.configure_alazar(
    records_per_buffer=1,
    buffers_per_acquisition=int(int_time/buffer_len), # set averaging by choosing the number of buffers
)
detector.setup() # this makes sure all params are set correctly

### Set up the measurement info
exp = select_experiment('resonator_spec', SAMPLE) # select experiment (info for the database)

# need to make sure the sweep is correctly described in the beginning!
meas = MeasurementExt(station, exp=exp, 
                      hard_sweep_detector=detector,
                      soft_sweep_params=[
                          hetsrc.frequency,
                          ro_attenuator.attenuation,
                      ])

# and finally, the measurement loop
meas.write_period = 2 # how often to write to the DB [s]
with meas.run() as datasaver:
    
    # this is to connect the database to the plottr.
    plot_subscriber = QcodesDatasetSubscriber(datasaver.dataset)
    datasaver.dataset.subscribe(plot_subscriber, state=[])
    
    # the loop itself is trivial
    for att in ro_atts:
        ro_attenuator.attenuation(att)
        
        for f in rf_frqs:
            hetsrc.frequency(f)
            time.sleep(0.01)

            detector.acquisition() # this gets the data from the alazar
            datasaver.add_result(*meas.get_result()) # this adds all data to the datasaver.

No ro_gate defined in channel map. No readout gate pulse will be generated.
No src_gate defined in channel map. No src gate pulse will be generated.


Starting experimental run with id: 1283


KeyboardInterrupt: 